# Census Income Analysis (Kohavi & Becker Adult Dataset)

This notebook answers each prompt from the assignment using clearly labeled sections.


## 1) Setup and data loading

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
import statsmodels.formula.api as smf

sns.set_theme(style="whitegrid")


In [ ]:
# Preferred source: local file if available; fallback to UCI URL.
cols = [
    "age", "workclass", "fnlwgt", "education", "education_num", "marital_status",
    "occupation", "relationship", "race", "sex", "capital_gain", "capital_loss",
    "hours_per_week", "native_country", "gross_income_group"
]

local_path = "adult.data"
uci_url = "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"

try:
    df = pd.read_csv(local_path, header=None, names=cols, na_values=" ?", skipinitialspace=True)
    print(f"Loaded local file: {local_path}")
except FileNotFoundError:
    df = pd.read_csv(uci_url, header=None, names=cols, na_values=" ?", skipinitialspace=True)
    print("Loaded dataset from UCI URL")

print(df.shape)
df.head()


## 2) Data exploration

### 2.1 Column checks and data types

In [ ]:
print(df.dtypes)


In [ ]:
# Optional: enforce expected dtypes explicitly
cat_cols = ["workclass", "education", "marital_status", "occupation", "relationship", "race", "sex", "native_country", "gross_income_group"]
for c in cat_cols:
    df[c] = df[c].astype("category")


### 2.2 Missing values

In [ ]:
# Missing values are represented as '?' (or ' ?' before stripping).
# They were converted to NaN during read_csv via na_values.
missing_counts = df.isna().sum().sort_values(ascending=False)
missing_counts


### 2.3 Distribution of capital_gain and capital_loss + transformation

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 4))
sns.histplot(df["capital_gain"], bins=40, ax=axes[0])
axes[0].set_title("capital_gain distribution")
sns.histplot(df["capital_loss"], bins=40, ax=axes[1])
axes[1].set_title("capital_loss distribution")
plt.tight_layout()


In [ ]:
# Suggested transformation: convert highly zero-inflated financial variables
# into categorical indicators with optional positive-level bins.
df["capital_gain_cat"] = pd.cut(
    df["capital_gain"],
    bins=[-1, 0, 5000, np.inf],
    labels=["none", "low", "high"]
)

df["capital_loss_cat"] = pd.cut(
    df["capital_loss"],
    bins=[-1, 0, 2000, np.inf],
    labels=["none", "low", "high"]
)

print(df["capital_gain_cat"].value_counts(dropna=False))
print(df["capital_loss_cat"].value_counts(dropna=False))


### 2.4 Distribution of fnlwgt, sex comparisons, and outliers

In [ ]:
df["fnlwgt"].describe(percentiles=[0.01, 0.05, 0.5, 0.95, 0.99])


In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 4))
sns.histplot(df["fnlwgt"], bins=50, ax=axes[0])
axes[0].set_title("fnlwgt overall")
sns.boxplot(data=df, x="sex", y="fnlwgt", ax=axes[1])
axes[1].set_title("fnlwgt by sex")
plt.tight_layout()


In [ ]:
# Outlier handling choice (if desired): mark top 1% as missing
cutoff = df["fnlwgt"].quantile(0.99)
df.loc[df["fnlwgt"] > cutoff, "fnlwgt"] = np.nan
print(f"Set fnlwgt > {cutoff:,.0f} to NaN")


## 3) Linear regression: hours_per_week vs sex (+ controls)

In [ ]:
# Model 1: hours_per_week ~ sex
m1 = smf.ols("hours_per_week ~ C(sex)", data=df).fit()
print(m1.summary())

# Model 2: add education_num
m2 = smf.ols("hours_per_week ~ C(sex) + education_num", data=df).fit()
print(m2.summary())

# Model 3: add income group
m3 = smf.ols("hours_per_week ~ C(sex) + education_num + C(gross_income_group)", data=df).fit()
print(m3.summary())


In [ ]:
# Compare models by adjusted R^2, AIC, and BIC
comparison = pd.DataFrame({
    "model": ["M1", "M2", "M3"],
    "adj_R2": [m1.rsquared_adj, m2.rsquared_adj, m3.rsquared_adj],
    "AIC": [m1.aic, m2.aic, m3.aic],
    "BIC": [m1.bic, m2.bic, m3.bic]
})
comparison


## 4) Correlation analysis

In [ ]:
subset = df[["age", "education_num", "hours_per_week"]].dropna()
corr = subset.corr()
corr


In [ ]:
# Significance tests for correlations with |r| > 0.1
pairs = [("age", "education_num"), ("age", "hours_per_week"), ("education_num", "hours_per_week")]
for x, y in pairs:
    r, p = stats.pearsonr(subset[x], subset[y])
    if abs(r) > 0.1:
        print(f"{x} vs {y}: r={r:.4f}, p={p:.4g}")


In [ ]:
# education_num vs age by sex
for s in df["sex"].dropna().unique():
    d = df.loc[df["sex"] == s, ["education_num", "age"]].dropna()
    r, p = stats.pearsonr(d["education_num"], d["age"])
    print(f"{s}: r={r:.4f}, p={p:.4g}, n={len(d)}")


In [ ]:
# Covariance matrix for education_num and hours_per_week
cov_mat = df[["education_num", "hours_per_week"]].dropna().cov()
cov_mat


## 5) Short written answers (fill after running)

- Data types mostly match expected schema; categorical fields should be set to category/object and numeric fields to integer.
- Missing values are represented as `?` and converted to `NaN`.
- `capital_gain` and `capital_loss` are highly right-skewed and zero-inflated, so categorical transformations are reasonable.
- `fnlwgt` is typically right-skewed, not symmetric; compare medians/quantiles by sex before deciding outlier treatment.
- For model selection among nested models, adjusted R² plus AIC/BIC is a practical choice.
